# CONNECT THE BOARD

In [2]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
f = open("main.py", "r")
print(f.read())

In [ ]:
%sendtofile lib/collector.py --source lib/collector.py
%sendtofile lib/peripherals.py --source lib/peripherals.py
%sendtofile lib/signal.py --source lib/signal.py

In [3]:
%sendtofile main.py --source main.py

Sent 116 lines (2881 bytes) to main.py.


# COLLECTOR METHOD

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

from lib.requests import MicroWebCli as requests
import ujson as json

from lib.utils import connect_wifi, load_env_vars

from ulab import numpy as np
import utime as time
from lib.collector import Collector 
from lib.runner import Runner 

In [ ]:
collector = Collector(buffer_size=256, preprocess=True, ds=True)

SPI_params= {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,
}

collector.setup(spi_params=SPI_params)

time.sleep(4)

collector.run()

time.sleep(10)

while True:
    time.sleep(1)
    bt = time.time()
    data = collector.read_output_buffer()
    et = time.time()
    print(et-bt,data)

In [ ]:
collector.stop()

In [ ]:
runner = Runner('CCA', buffer_size=256)
runner.setup()

time.sleep(4)
runner.run()
while True:
    time.sleep(1)
    data = runner.read_output_buffer()
    print(data)

In [ ]:
runner.stop()

# DIRECTLY READ ADC, SKIP EVERYTHING ELSE

In [ ]:
%serialconnect to --port="COM5" --baud=115200

# TESTING GAIN

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)
cs.off() #active low
print("Set GPIO 5 LOW")
spi.write(data)
print("Sent gain")
cs.on() #active high
print("Set GPIO 5 HIGH")
while True:
    time.sleep(1)
    print(adc.read())

# Testing live decoding

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)

num_train = 6

#send 7hz data
for i in range(num_train):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"7":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(num_train):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"10":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(num_train):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"12":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

# For board

In [25]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
# data = bytearray(100)
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256
fs = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=fs, callback=sample_callback)

for i in range(30):
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

connecting to network...
network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '194.168.4.100')
70208
{'raw_data': [1104, 1705, 2382, 2251, 1475, 1068, 1601, 2320, 2313, 1535, 1061, 2143, 2274, 2339, 1615, 1067, 1414, 2190, 2382, 1684, 1078, 1334, 2206, 2407, 1781, 1121, 1258, 2054, 2426, 1905, 1205, 1230, 2009, 2438, 1986, 1264, 1181, 1936, 2446, 2092, 1343, 1136, 1840, 2425, 2160, 1407, 1110, 1741, 2391, 2217, 1452, 1101, 1665, 2357, 2293, 1515, 1072, 1557, 2282, 2333, 1584, 1061, 1453, 2203, 2345, 1674, 1101, 1387, 2157, 2404, 1756, 1116, 1296, 2086, 2412, 1264, 1080, 1250, 2023, 2432, 1939, 1235, 1189, 1968, 2448, 2047, 1310, 1139, 1878, 2421, 1803, 1361, 1125, 1808, 2425, 2195, 1424, 1086, 1691, 2367, 2256, 1483, 1074, 1590, 2297, 2289, 1534, 1079, 1517, 2256, 2353, 1617, 1060, 1402, 2161, 2371, 1475, 1103, 1342, 2128, 2426, 1840, 1166, 1276, 2399, 2394, 1904, 1213, 1263, 1993, 2443, 2013, 1293, 1188, 1939, 2455, 2096, 1350, 1152, 1853, 2445, 2179, 1115, 1114, 1742, 239

In [11]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 

>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


# 64hz sampling over 4s window

In [11]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256
sampling_rate = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=sampling_rate, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)

decode_period = 4
#send 7hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"7":data}
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"10":data}
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"12":data}
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"raw_data":data}
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

connecting to network...
network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '194.168.4.100')
........................

*** Sending Ctrl-C



Keyboard interrupt while waiting response on Ctrl-C



In [34]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on() 

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256*4
sampling_rate = 256

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=sampling_rate, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)

decode_period = 4
#send 7hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"7":data[::4]}
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"10":data[::4]}
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"12":data[::4]}
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(decode_period)
    data = adc_sample
    toSend = {"raw_data":data[::4]}
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '194.168.4.100')
......................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 110, in <module>
KeyboardInterrupt: 


In [71]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [69]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json
from lib.signal import sos_filter
from ulab import numpy as np

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

data = bytearray([17,DEFAULT_SPI_PARAMS['output_amp_gain']])
cs = machine.Pin(5, machine.Pin.OUT)

# have to turn GPIO 5 off before writing the gain

cs.off() 
spi.write(data)
cs.on()

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 256*4
sampling_rate = 256

def preprocess_data(signal):

    ds_factor = 4
    signal = np.array(signal) - np.mean(signal)
    return sos_filter(signal, fs=256)[::ds_factor]

    
def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=sampling_rate, callback=sample_callback)

requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 7 HZ ###"})
time.sleep(10)

decode_period = 4
#send 7hz data
for i in range(4):
    time.sleep(decode_period)
    data = preprocess_data(adc_sample).tolist()
    toSend = {"7":data}
    requests.JSONRequest("http://192.168.0.37:5001/7hz", toSend)
    print(gc.mem_free())

gc.collect()
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 10 HZ ###"})
time.sleep(10)

#send 10hz data
for i in range(4):
    time.sleep(decode_period)
    data = preprocess_data(adc_sample).tolist()
    toSend = {"10":data}
    requests.JSONRequest("http://192.168.0.37:5001/10hz", toSend)
    print(gc.mem_free())
    
gc.collect()
requests.JSONRequest("http://192.168.0.37:5001/message", {"message": "### LOOK AT 12 HZ ###"})
time.sleep(5)

#send 12hz data
for i in range(4):
    time.sleep(decode_period)
    data = preprocess_data(adc_sample).tolist()
    toSend = {"12":data}
    requests.JSONRequest("http://192.168.0.37:5001/12hz", toSend)

gc.collect()
requests.GETRequest("http://192.168.0.37:5001/isCalibrated")

while True:
    time.sleep(decode_period)
    data = preprocess_data(adc_sample).tolist()
    toSend = {"raw_data":data}
    requests.JSONRequest("http://192.168.0.37:5001/decode", toSend)

connecting to network...
network config: ('192.168.0.22', '255.255.255.0', '192.168.0.1', '194.168.4.100')
...48864
.34384
.34352
34400
...66192
.66608
.66576
.66560
...........

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 125, in <module>
KeyboardInterrupt: 


# Gain testing

In [6]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 

>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [9]:
%sendtofile main.py --source main.py

Sent 209 lines (4700 bytes) to main.py.


In [ ]:
f = open("main.py", "r")
print(f.read())

In [8]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# setting gain #

cs = machine.Pin(5, machine.Pin.OUT)

# have to pull GPIO 5 LOW before writing the gain

# will change lol
ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

# set for frequency the adc should be read and set the size of the array to send every t seconds 
pot_size = 512

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) >= pot_size:
        # fifo
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=pot_size, callback=sample_callback)

time.sleep(5)

for i in range(10):
    
    data = bytearray([17,0])

    cs.off() 
    spi.write(data)
    cs.on() 
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,50])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,100])

    cs.off() 
    spi.write(data)
    cs.on() 
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,150])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,200])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,220])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)
    
for i in range(10):
    
    data = bytearray([17,240])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

for i in range(10):
    
    data = bytearray([17,255])

    cs.off() 
    spi.write(data)
    cs.on()
    
    # set send time
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)
    
requests.GETRequest("http://192.168.0.37:5001/save")

network config: ('192.168.0.88', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.10784
{'raw_data': [1671, 1584, 1439, 1430, 1521, 1717, 1919, 2074, 2106, 2007, 1825, 1615, 1441, 1410, 1478, 1661, 1873, 2066, 2130, 2069, 1889, 1681, 1424, 1423, 1462, 1620, 1823, 2025, 2122, 2093, 1942, 1729, 1520, 1425, 1440, 1579, 1790, 2000, 2129, 2129, 1998, 1798, 1584, 1442, 1431, 1529, 1722, 1934, 2096, 2115, 2017, 1822, 1618, 1439, 1420, 1493, 1671, 1887, 2064, 2121, 2045, 1868, 1661, 1470, 1412, 1458, 1625, 1829, 2020, 2106, 2076, 1919, 1710, 1501, 1451, 1458, 1584, 1781, 1987, 2110, 2096, 1966, 1765, 1553, 1425, 1431, 1535, 1728, 1946, 2102, 2113, 2016, 1814, 1599, 1433, 1408, 1487, 1671, 1880, 2065, 2119, 2047, 1866, 1652, 1461, 1408, 1458, 1631, 1837, 2028, 2112, 2083, 1919, 1701, 1495, 1405, 1429, 1574, 1782, 1991, 2106, 2102, 1971, 1769, 1565, 1433, 1433, 1658, 1748, 1955, 2096, 2112, 2011, 1812, 1600, 1429, 1406, 1494, 1687, 1899, 2070, 2123, 2064, 1872, 1668, 1481, 1424, 1477, 1641, 185

12080
{'raw_data': [2118, 2121, 2039, 1859, 1647, 1463, 1417, 1479, 1649, 1859, 2043, 2118, 2070, 1901, 1686, 1488, 1457, 1470, 1610, 1811, 2020, 2127, 2111, 1968, 1761, 1553, 1429, 1438, 1570, 1755, 1966, 2093, 2102, 1990, 1787, 1584, 1438, 1525, 1551, 1731, 1934, 2100, 2128, 2044, 1851, 1631, 1451, 1411, 1483, 1662, 1872, 2068, 2139, 2087, 1917, 1702, 1505, 1424, 1458, 1616, 1815, 2015, 2125, 2099, 1961, 1759, 1545, 1431, 1459, 1571, 1762, 1968, 2103, 2098, 1984, 1779, 1571, 1427, 1418, 1511, 1708, 1923, 2091, 2121, 2032, 1847, 1637, 1461, 1411, 1685, 1711, 1874, 2067, 2125, 2071, 1904, 1697, 1505, 1435, 1478, 1628, 1839, 2032, 2125, 2098, 1940, 1731, 1521, 1411, 1439, 1571, 1772, 1981, 2110, 2109, 1986, 1790, 1571, 1429, 1426, 1578, 1723, 1926, 2085, 2109, 2022, 1821, 1615, 1446, 1413, 1494, 1677, 1873, 2046, 2109, 2035, 1856, 1648, 1465, 1404, 1461, 1636, 1949, 2055, 2130, 2094, 1933, 1713, 1498, 1399, 1431, 1574, 1776, 1983, 2103, 2102, 1968, 1757, 1551, 1413, 1413, 1525, 1730, 19

14688
{'raw_data': [1470, 1429, 1429, 1543, 1734, 1942, 2096, 2109, 2007, 1808, 1611, 1456, 1441, 1523, 1712, 1913, 2128, 2121, 2042, 1857, 1645, 1471, 1415, 1475, 1650, 1857, 2047, 2117, 1970, 1902, 1687, 1488, 1409, 1456, 1602, 1813, 2010, 2128, 2106, 1959, 1750, 1543, 1427, 1449, 1569, 1774, 1983, 2113, 2124, 2005, 1808, 1594, 1439, 1428, 1520, 1714, 1923, 2087, 2058, 2024, 1840, 1619, 1439, 1407, 1471, 1648, 1860, 2045, 2119, 2074, 1900, 1686, 1488, 1415, 1453, 1614, 1815, 2014, 2118, 2096, 1776, 1733, 1520, 1406, 1418, 1547, 1743, 1949, 2087, 2100, 1979, 1773, 1562, 1411, 1401, 1505, 1706, 1920, 2096, 2129, 2059, 1857, 1635, 1457, 1419, 1483, 1661, 1869, 2059, 2121, 2064, 1855, 1667, 1479, 1415, 1457, 1615, 1824, 2032, 2126, 2109, 1956, 1751, 1546, 1431, 1454, 1571, 1766, 1968, 2097, 2099, 1979, 1598, 1554, 1434, 1426, 1526, 1710, 1905, 2070, 2085, 1955, 1795, 1591, 1429, 1402, 1474, 1658, 1877, 2064, 2122, 2060, 1885, 1664, 1463, 1394, 1440, 1602, 1810, 2009, 2113, 2091, 1946, 16

17088
{'raw_data': [2001, 2096, 2087, 1943, 1744, 1529, 1416, 1438, 1559, 1748, 1959, 2110, 2043, 1994, 1797, 1583, 1419, 1423, 1505, 1705, 1911, 2093, 2139, 2064, 1872, 1662, 1475, 1424, 1477, 1641, 1850, 2043, 2125, 2079, 1915, 1722, 1520, 1431, 1471, 1613, 1807, 2000, 2113, 2091, 1947, 1745, 1520, 1420, 1430, 1555, 1755, 1959, 2105, 2113, 2003, 1794, 1584, 1435, 1420, 1519, 1703, 1905, 2079, 2115, 2032, 1843, 1641, 1463, 1519, 1535, 1664, 1869, 2059, 2123, 2074, 1891, 1685, 1501, 1425, 1458, 1607, 1808, 2000, 2096, 2073, 1929, 1718, 1514, 1414, 1429, 1562, 1763, 1975, 2106, 1679, 1653, 1635, 1566, 1418, 1401, 1493, 1685, 1894, 2063, 2101, 2026, 1837, 1633, 1461, 1424, 1497, 1678, 1883, 2064, 2128, 2064, 1884, 1674, 1479, 1409, 1456, 1702, 1909, 2038, 2128, 2089, 1950, 1737, 1519, 1406, 1427, 1556, 1757, 1965, 2096, 2093, 1977, 1767, 1557, 1419, 1413, 1518, 1712, 1922, 2090, 2119, 2023, 1838, 1627, 1450, 1417, 1475, 1682, 1861, 2034, 2101, 2037, 1874, 1675, 1488, 1427, 1482, 1643, 18

19616
{'raw_data': [1502, 1917, 1952, 2099, 2155, 2078, 1884, 1653, 1454, 1390, 1442, 1606, 1819, 2029, 2127, 2081, 1756, 1694, 1499, 1408, 1451, 1599, 1808, 2023, 2149, 2128, 1980, 1755, 1543, 1410, 1413, 1535, 1749, 1968, 2123, 2144, 2032, 1817, 1424, 1415, 1395, 1482, 1687, 1904, 2090, 2142, 2071, 1872, 1651, 1461, 1411, 1483, 1662, 1866, 2063, 2143, 2093, 1904, 1684, 1470, 1388, 1440, 1599, 1808, 2023, 2129, 2110, 1950, 1733, 1488, 1393, 1424, 1553, 1765, 1983, 2122, 2126, 2002, 1789, 1566, 1400, 1391, 1500, 1698, 1922, 2094, 2126, 2030, 1835, 1616, 1383, 1391, 1457, 1642, 1856, 2062, 2129, 2082, 1892, 1680, 1470, 1392, 1451, 1613, 1824, 2031, 2130, 2100, 1938, 1712, 1494, 1379, 1397, 1535, 1762, 1982, 2125, 2128, 2001, 1789, 1574, 1405, 1419, 1519, 1719, 1931, 2099, 2128, 2031, 1813, 1590, 1409, 1379, 1461, 1659, 1877, 2070, 2135, 2065, 1878, 1653, 1447, 1373, 1548, 1611, 1833, 2042, 2145, 2114, 1947, 1724, 1505, 1403, 1421, 1552, 1765, 1983, 2117, 2121, 1984, 1773, 1551, 1422, 14

...

{'raw_data': [1587, 1463, 1395, 1546, 1621, 1838, 2067, 2154, 2096, 1931, 1707, 1489, 1376, 1408, 1569, 1787, 2003, 2141, 2129, 1984, 1766, 1399, 1379, 1389, 1509, 1725, 1951, 2118, 2149, 2033, 1821, 1584, 1407, 1385, 1462, 1664, 1899, 2096, 2151, 2079, 1872, 1646, 1441, 1386, 1443, 1621, 1847, 2064, 2155, 2105, 1939, 1520, 1477, 1387, 1424, 1581, 1800, 2026, 2152, 2130, 1984, 1761, 1527, 1392, 1393, 1512, 1729, 1951, 2121, 2138, 2023, 1810, 1586, 1383, 1389, 1483, 1681, 1901, 2085, 2141, 2059, 1856, 1632, 1444, 1388, 1453, 1641, 1856, 2063, 2138, 2079, 1895, 1671, 1454, 1363, 1406, 1581, 1805, 2023, 2150, 2128, 1968, 1761, 1523, 1390, 1404, 1543, 1757, 1987, 2135, 2143, 2017, 1794, 1570, 1395, 1378, 1477, 1690, 1919, 2096, 2136, 2053, 1843, 1621, 1427, 1375, 1509, 1661, 1881, 2090, 2177, 2112, 1921, 1694, 1478, 1389, 1437, 1607, 1822, 2038, 2147, 2128, 1968, 1744, 1519, 1399, 1415, 1563, 1776, 2000, 2150, 2157, 2017, 1799, 1579, 1413, 1440, 1515, 1719, 1945, 2123, 2160, 2064, 1851, 16

{'raw_data': [1675, 1851, 2123, 2154, 2097, 1872, 1701, 1481, 1392, 1437, 1587, 1799, 2010, 2117, 2098, 1779, 1712, 1505, 1404, 1424, 1566, 1778, 1991, 2128, 2127, 1998, 1777, 1551, 1397, 1391, 1491, 1709, 1930, 2101, 2134, 2031, 1825, 1415, 1407, 1392, 1465, 1653, 1899, 2086, 2146, 2078, 1885, 1659, 1447, 1379, 1429, 1610, 1838, 2059, 2164, 2133, 1968, 1743, 1506, 1399, 1418, 1560, 1777, 2000, 2144, 2146, 2010, 1787, 1497, 1393, 1389, 1493, 1707, 1929, 2108, 2145, 2047, 1835, 1614, 1424, 1393, 1477, 1680, 1902, 2103, 2165, 2098, 1905, 1669, 1459, 1398, 1443, 1616, 1836, 2056, 2143, 2117, 1945, 1717, 1488, 1377, 1399, 1551, 1760, 1983, 2128, 2128, 1987, 1774, 1554, 1409, 1409, 1533, 1749, 1971, 2145, 2166, 2065, 1846, 1618, 1391, 1394, 1473, 1680, 1903, 2098, 2167, 2096, 1888, 1663, 1451, 1374, 1434, 1612, 1837, 2060, 2159, 2126, 1949, 1723, 1502, 1394, 1579, 1602, 1802, 2009, 2148, 2145, 2000, 1744, 1551, 1403, 1408, 1523, 1738, 1961, 2128, 2144, 2035, 1822, 1598, 1410, 1371, 1463, 16

{'raw_data': [1584, 1722, 1943, 2107, 2130, 1935, 1808, 1584, 1411, 1394, 1488, 1687, 1906, 2158, 2156, 2067, 1867, 1642, 1438, 1382, 1452, 1648, 1866, 2085, 2170, 2119, 1936, 1709, 1483, 1386, 1421, 1578, 1791, 2011, 2127, 2111, 1967, 1733, 1503, 1378, 1392, 1520, 1745, 1969, 2148, 2151, 2034, 1825, 1601, 1424, 1401, 1495, 1701, 1906, 2097, 2146, 2064, 1859, 1643, 1439, 1392, 1459, 1647, 1872, 2080, 2083, 2069, 1930, 1703, 1475, 1382, 1424, 1598, 1811, 2033, 2160, 2131, 1980, 1750, 1520, 1381, 1402, 1534, 1753, 1984, 2141, 2159, 2039, 1822, 1587, 1417, 1392, 1490, 1695, 1924, 2110, 2130, 2064, 1850, 1619, 1421, 1355, 1434, 1627, 1840, 2063, 2145, 2108, 1923, 1696, 1478, 1385, 1424, 1584, 1793, 2011, 2128, 2117, 1851, 1744, 1520, 1393, 1423, 1552, 1767, 1981, 2133, 2147, 2019, 1804, 1579, 1419, 1405, 1511, 1717, 1938, 2130, 2170, 2071, 1859, 1625, 1427, 1375, 1450, 1645, 1875, 2094, 2173, 2008, 1914, 1711, 1483, 1397, 1439, 1607, 1819, 2035, 2151, 2121, 1965, 1745, 1513, 1383, 1401, 15

{'raw_data': [1872, 2032, 2134, 2053, 1935, 1455, 1437, 1381, 1408, 1555, 1790, 2001, 2128, 2128, 1983, 1769, 1535, 1381, 1391, 1503, 1728, 1963, 2134, 2160, 2064, 1849, 1619, 1424, 1379, 1463, 1662, 1886, 2081, 2149, 2086, 1898, 1573, 1471, 1406, 1456, 1639, 1853, 2067, 2160, 2110, 1940, 1712, 1487, 1386, 1423, 1569, 1793, 2022, 2160, 2154, 2015, 1795, 1563, 1383, 1418, 1520, 1715, 1954, 2110, 2139, 2037, 1835, 1613, 1426, 1393, 1481, 1680, 1904, 2097, 2147, 2079, 1883, 1657, 1454, 1389, 1445, 1623, 1850, 2064, 2154, 2113, 1937, 1706, 1383, 1375, 1402, 1566, 1786, 2010, 2149, 2138, 1999, 1776, 1550, 1398, 1405, 1531, 1747, 1979, 2146, 2171, 2063, 1839, 1616, 1426, 1474, 1490, 1680, 1904, 2095, 2153, 2064, 1859, 1625, 1419, 1360, 1419, 1609, 1838, 2061, 2160, 2115, 1943, 1712, 1484, 1376, 1406, 1556, 1777, 2000, 2135, 2126, 1983, 1749, 1519, 1375, 1379, 1489, 1710, 1942, 2112, 2146, 2047, 1833, 1605, 1419, 1391, 1482, 1675, 1893, 2094, 2147, 2074, 1867, 1639, 1424, 1367, 1426, 1727, 18

{'raw_data': [2078, 2000, 1777, 1552, 1401, 1399, 1514, 1868, 1967, 2124, 2129, 2031, 1819, 1602, 1431, 1462, 1498, 1697, 1919, 2116, 2171, 2093, 1884, 1659, 1441, 1385, 1441, 1616, 1840, 2058, 2145, 2101, 1923, 1698, 1483, 1381, 1419, 1835, 1858, 2063, 2130, 2112, 1942, 1736, 1504, 1367, 1388, 1520, 1741, 1971, 2143, 2160, 2047, 1837, 1591, 1414, 1387, 1472, 1673, 1904, 2101, 2155, 2080, 1872, 1655, 1456, 1397, 1462, 1675, 1872, 2080, 2165, 2112, 1929, 1695, 1469, 1378, 1409, 1570, 1795, 2012, 2138, 2121, 1968, 1744, 1519, 1382, 1392, 1520, 2026, 2047, 2131, 2139, 2021, 1805, 1573, 1405, 1392, 1488, 1696, 1927, 2113, 2160, 2074, 1863, 1630, 1421, 1362, 1436, 1622, 1855, 2067, 2157, 2111, 1926, 1703, 1484, 1391, 1430, 1594, 1909, 2045, 2158, 2138, 1986, 1761, 1531, 1393, 1398, 1522, 1741, 1968, 2119, 2128, 2010, 1797, 1577, 1409, 1392, 1488, 1686, 1914, 2131, 2129, 2001, 1821, 1602, 1407, 1361, 1437, 1629, 1853, 2059, 2138, 2089, 1897, 1675, 1456, 1363, 1415, 1582, 1803, 2029, 2145, 21